In [1]:
import pandas as pd
import numpy as np
import re

f = open("data/data19.txt", "r")
lines = [line[:-1] for line in f.readlines()]

lines = [
    "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.",
    "Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."
]

def get_costs(line):
    d = {}
    k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = tuple([int(z) for z in re.findall(r'\d+', line)])
    return k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs

print(len(lines))
print(get_costs(lines[0]))

2
(1, 4, 2, 3, 14, 2, 7)


In [2]:
k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = get_costs(lines[0])
print(k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs)

def get_maxperf(t, ore, clay, obs, geo, n_ore, n_clay, n_obs, n_geo):

    if t==0:
        return geo#, []
    
    res = []
    #strat = []
    ore2 = ore + n_ore
    clay2 = clay + n_clay
    obs2 = obs + n_obs
    geo2 = geo + n_geo    

    #r, s0 = get_maxperf(t-1, ore2, clay2, obs2, geo2, n_ore, n_clay, n_obs, n_geo)
    res.append(get_maxperf(t-1, ore2, clay2, obs2, geo2, n_ore, n_clay, n_obs, n_geo))
    #strat.append(["wait"] + s0)
    if ore >= c_ore_ore:
        #r, s1 = get_maxperf(t-1, ore2 - c_ore_ore, clay2, obs2, geo2, n_ore + 1, n_clay, n_obs, n_geo)
        res.append(get_maxperf(t-1, ore2 - c_ore_ore, clay2, obs2, geo2, n_ore + 1, n_clay, n_obs, n_geo))
        #strat.append(["r_ore"] + s1)
    if ore >= c_clay_ore:
        #r, s2 = get_maxperf(t-1, ore2 - c_clay_ore, clay2, obs2, geo2, n_ore, n_clay + 1, n_obs, n_geo)
        res.append(get_maxperf(t-1, ore2 - c_clay_ore, clay2, obs2, geo2, n_ore, n_clay + 1, n_obs, n_geo))
        #strat.append(["r_clay"] + s2)
    if (ore >= c_obs_ore) and (clay >= c_obs_clay):
        #r, s3 = get_maxperf(t-1, ore2 - c_obs_ore, clay2 - c_obs_clay, obs2, geo2, n_ore, n_clay, n_obs + 1, n_geo)
        res.append(get_maxperf(t-1, ore2 - c_obs_ore, clay2 - c_obs_clay, obs2, geo2, n_ore, n_clay, n_obs + 1, n_geo))
        #strat.append(["r_obs"] + s3)
    if (ore >= c_geo_ore) and (obs >= c_geo_obs):
        #r, s4 = get_maxperf(t-1, ore2 - c_geo_ore, clay2 , obs2 - c_geo_obs, geo2, n_ore, n_clay, n_obs, n_geo + 1)
        res.append(get_maxperf(t-1, ore2 - c_geo_ore, clay2 , obs2 - c_geo_obs, geo2, n_ore, n_clay, n_obs, n_geo + 1))
        #strat.append(["r_geo"] + s4)
    
    m = max(res)
    #imax = res.index(max(res))

    return m#, strat[imax]

print(get_maxperf(10, 0, 0, 0, 0, 1, 0, 0, 0))


1 4 2 3 14 2 7
0


In [3]:
get_maxperf(4, 4, 25, 7, 2, 1, 4, 2, 1)


9

In [4]:
get_maxperf(14, 4, 15, 0, 0, 1, 3, 0, 0)


9

In [16]:
lines = [
    "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.",
    "Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."
]


def update(x):
    y = x.copy()
    y[:4] += y[4:]
    return y

L_states = [np.array([0, 0, 0, 0, 1, 0, 0, 0], dtype='int8')]

tmax = 24
def get_options(line, tmax, verbose=True):
    _, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = get_costs(line)
    r_ore = np.array([- c_ore_ore, 0, 0, 0, 1, 0, 0, 0], dtype='int8')
    r_clay = np.array([- c_clay_ore, 0, 0, 0, 0, 1, 0, 0], dtype='int8')
    r_obs = np.array([- c_obs_ore, - c_obs_clay, 0, 0, 0, 0, 1, 0], dtype='int8')
    r_geo = np.array([-c_geo_ore, 0, - c_geo_obs, 0, 0, 0, 0, 1], dtype='int8')
    L_states = [np.array([0, 0, 0, 0, 1, 0, 0, 0], dtype='int8')]
    maxore = max([c_ore_ore, c_clay_ore, c_obs_ore, c_geo_ore])

    for t in range(1, tmax+1):
        list_options = []
        for x in L_states:
            y = update(x)
            n_choices = 0
            if (x[0] >= c_geo_ore) and (x[2] >= c_geo_obs):
                list_options.append(y+r_geo)
                n_choices += 2
            if (n_choices<2) and (x[0] >= c_obs_ore) and (x[1] >= c_obs_clay):
                list_options.append(y+r_obs)
                n_choices += 1
            if (n_choices<2) and (x[0] >= c_clay_ore):
                list_options.append(y+r_clay)
                n_choices += 1
            if (n_choices<2) and (x[0] >= c_ore_ore):
                list_options.append(y+r_ore)
                n_choices += 0
            if n_choices<2:
                list_options.append(y)
        m = max([l[3] for l in list_options])
        L_1 = [l for l in list_options if l[3]>m-2]
        L_states = [l for l in L_1 if l[0]<2*maxore]
        if verbose:
            print(t, len(L_1), len(L_states))
    return L_states

l1 = get_options(lines[0], tmax)
#l2 = get_options(lines[1], tmax)
max([s[3] for s in l1])#, max([s[3] for s in l2])

1 1 1
2 1 1
3 2 2
4 3 3
5 7 7
6 12 12
7 29 29
8 57 56
9 140 138
10 305 288
11 743 693
12 1656 1447
13 3601 3027
14 7148 5789
15 12897 10531
16 24503 19796
17 44750 36063
18 78780 65048
19 140126 116153
20 2709 2599
21 5210 4987
22 392 392
23 547 535
24 773 748


9

In [17]:
f = open("data/data19.txt", "r")
lines = [line[:-1] for line in f.readlines()]
tmax = 24

best_k = {}
for i, line in enumerate(lines):
    options = get_options(line, 24, False)
    best_k[i+1] = max([s[3] for s in options])
    print(i+1, best_k[i+1])
s = 0
for i in best_k.keys():
    s += i*best_k[i]
print(s) # 1273 is too low

1 3
2 13
3 0
4 0
5 0
6 8
7 0
8 6
9 0
10 2
11 0
12 0
13 1
14 12
15 0
16 0
17 0
18 0
19 0
20 0
21 6
22 1
23 0
24 15
25 1
26 3
27 7
28 1
29 1
30 3
1273


In [7]:
#f = open("data/data19.txt", "r")
#lines = [line[:-1] for line in f.readlines()]

lines = [
    "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.",
    "Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."
]

def get_options(line, tmax=24, verbose=True):
    _, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = get_costs(line)
    maxore = max([c_ore_ore, c_clay_ore, c_obs_ore, c_geo_ore])
    maxclay = c_obs_clay
    maxobs = c_geo_obs
    L_states = [(0, 0, 0, 0, 1, 0, 0, 0)]
    for t in range(1, tmax+1):
        list_options = []
        for x in L_states:
            y = (x[0]+x[4], x[1]+x[5], x[2]+x[6], x[3]+x[7], x[4], x[5], x[6], x[7])
            n_choices = 0
            if (x[0] >= c_geo_ore) and (x[2] >= c_geo_obs):
                list_options.append((y[0]-c_geo_ore, y[1], y[2]-c_geo_obs, y[3], y[4], y[5], y[6], y[7]+1))
                n_choices += 2
            if (n_choices<2) and (x[0] >= c_obs_ore) and (x[1] >= c_obs_clay):
                list_options.append((y[0]-c_obs_ore, y[1]-c_obs_clay, y[2], y[3], y[4], y[5], y[6]+1, y[7]))
                n_choices += 1
            if (n_choices<2) and (x[0] >= c_clay_ore):
                list_options.append((y[0]-c_clay_ore, y[1], y[2], y[3], y[4], y[5]+1, y[6], y[7]))
                n_choices += 1
            if (n_choices<2) and (x[0] >= c_ore_ore):
                list_options.append((y[0]-c_ore_ore, y[1], y[2], y[3], y[4]+1, y[5], y[6], y[7]))
                n_choices += 1
            if n_choices<2:
                list_options.append(y)
        m = max([x[3] for x in list_options])
        L_states = [x for x in list_options if (x[3]>m-3)]
        if verbose:
            print(t, len(list_options), len(L_states))
    return L_states

l1 = get_options(lines[0], 32, False)
print(max([s[3] for s in l1]))

l2 = get_options(lines[1], 32, False)
print(max([s[3] for s in l2]))

56
62


In [8]:
f = open("data/data19.txt", "r")
lines = [line[:-1] for line in f.readlines()]
tmax = 32

best_k = {}
for i, line in enumerate(lines[:3]):
    options = get_options(line, tmax, False)
    best_k[i+1] = max([s[3] for s in options])
    print(i+1, best_k[i+1])
#s = 0
#for i in best_k.keys():
#    s += i*best_k[i]
#print(s) # 1335 is too low

1 35
2 72
3 11


In [9]:
35*72*11

27720

In [13]:
pd.set_option('display.max_columns', 50)
#df = pd.DataFrame(columns=list(range(1, 31))+['res'])
#df.loc[len(df), :30] = list(best_k.keys())

df.loc[len(df), :30] = list(best_k.values())
df.loc[len(df)-1, 'res'] = (df.loc[0, :] * df.loc[len(df)-1, :]).sum()
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,res
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,NaN
1,3,13,0,0,0,10,0,7,0,2,0,0,1,12,1,0,0,0,0,0,7,1,0,15,1,3,8,1,1,3,1356
2,3,13,0,0,0,10,0,7,0,2,0,0,1,12,1,0,0,0,0,0,6,1,0,15,1,3,8,0,1,3,1307


In [76]:
s = df.iloc[1:, :-1].max(axis=0)
res = 0
for i in s.index:
    res += int(i)*s[i]
print(res)

1335


In [2]:
import re
import pulp as lp
tmax = 32

lines = [
    "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.",
    "Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."
]

#f = open("data/data19.txt", "r")
#lines = [line[:-1] for line in f.readlines()]

best_k = {}

for k, line in enumerate(lines):
    _, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = get_costs(line)

    problem = lp.LpProblem("problem_{:02d}".format(k), lp.LpMaximize)
    robots_geo = []
    for i in range(tmax):
        robots_geo.append(lp.LpVariable("robots_geo{:02d}".format(i), 0, 0 if i == 0 else None, lp.LpInteger))

    problem += robots_geo[0] + \
        robots_geo[1] + \
        robots_geo[2] + \
        robots_geo[3] + \
        robots_geo[4] + \
        robots_geo[5] + \
        robots_geo[6] + \
        robots_geo[7] + \
        robots_geo[8] + \
        robots_geo[9] + \
        robots_geo[10] + \
        robots_geo[11] + \
        robots_geo[12] + \
        robots_geo[13] + \
        robots_geo[14] + \
        robots_geo[15] + \
        robots_geo[16] + \
        robots_geo[17] + \
        robots_geo[18] + \
        robots_geo[19] + \
        robots_geo[20] + \
        robots_geo[21] + \
        robots_geo[22] + \
        robots_geo[23] + \
        robots_geo[24] + \
        robots_geo[25] + \
        robots_geo[26] + \
        robots_geo[27] + \
        robots_geo[28] + \
        robots_geo[29] + \
        robots_geo[30] + \
        robots_geo[31]

    build_ore = []
    build_cly = []
    build_obs = []
    build_geo = []

    onhand_ore = []
    onhand_cly = []
    onhand_obs = []

    robots_ore = []
    robots_cly = []
    robots_obs = []

    for i in range(tmax):
        if i == 0:
            robots_ore.append(lp.LpVariable("robots_ore{:02d}".format(i), 1, 1, lp.LpInteger))
            robots_cly.append(lp.LpVariable("robots_cly{:02d}".format(i), 0, 0, lp.LpInteger))
            robots_obs.append(lp.LpVariable("robots_obs{:02d}".format(i), 0, 0, lp.LpInteger))
            onhand_ore.append(lp.LpVariable("onhand_ore{:02d}".format(i), 0, 0, lp.LpInteger))
            onhand_cly.append(lp.LpVariable("onhand_cly{:02d}".format(i), 0, 0, lp.LpInteger))
            onhand_obs.append(lp.LpVariable("onhand_obs{:02d}".format(i), 0, 0, lp.LpInteger))
        else:
            robots_ore.append(lp.LpVariable("robots_ore{:02d}".format(i), 0, None, lp.LpInteger))
            robots_cly.append(lp.LpVariable("robots_cly{:02d}".format(i), 0, None, lp.LpInteger))
            robots_obs.append(lp.LpVariable("robots_obs{:02d}".format(i), 0, None, lp.LpInteger))
            onhand_ore.append(lp.LpVariable("onhand_ore{:02d}".format(i), 0, None, lp.LpInteger))
            onhand_cly.append(lp.LpVariable("onhand_cly{:02d}".format(i), 0, None, lp.LpInteger))
            onhand_obs.append(lp.LpVariable("onhand_obs{:02d}".format(i), 0, None, lp.LpInteger))
        build_ore.append(lp.LpVariable("build_ore{:02d}".format(i), 0, 1, lp.LpInteger))
        build_cly.append(lp.LpVariable("build_cly{:02d}".format(i), 0, 1, lp.LpInteger))
        build_obs.append(lp.LpVariable("build_obs{:02d}".format(i), 0, 1, lp.LpInteger))
        build_geo.append(lp.LpVariable("build_geo{:02d}".format(i), 0, 1, lp.LpInteger))
            
        problem += build_ore[-1] + build_cly[-1] + build_obs[-1] + build_geo[-1] <= 1

        if i != 0:
            # don't spend unavailable resources constraint
            problem += onhand_ore[-2] - build_ore[-1] * c_ore_ore >= 0
            problem += onhand_ore[-2] - build_cly[-1] * c_clay_ore >= 0
            problem += onhand_ore[-2] - build_obs[-1] * c_obs_ore >= 0
            problem += onhand_cly[-2] - build_obs[-1] * c_obs_clay >= 0
            problem += onhand_ore[-2] - build_geo[-1] * c_geo_ore >= 0
            problem += onhand_obs[-2] - build_geo[-1] * c_geo_obs >= 0

            # manage resources onhand constraint
            problem += onhand_ore[-1] == onhand_ore[-2] + robots_ore[-2] \
                - build_ore[-1] * c_ore_ore \
                    - build_cly[-1] * c_clay_ore \
                        - build_obs[-1] * c_obs_ore \
                            - build_geo[-1] * c_geo_ore
            problem += onhand_cly[-1] == onhand_cly[-2] + robots_cly[-2] \
                - build_obs[-1] * c_obs_clay
            problem += onhand_obs[-1] == onhand_obs[-2] + robots_obs[-2] \
                - build_geo[-1] * c_geo_obs

            # manage number of robots constraint
            problem += robots_ore[-1] == robots_ore[-2] + build_ore[-1]
            problem += robots_cly[-1] == robots_cly[-2] + build_cly[-1]
            problem += robots_obs[-1] == robots_obs[-2] + build_obs[-1]
            problem += robots_geo[i] == robots_geo[i-1] + build_geo[-1]

    status = problem.solve()
    res = problem.objective.value()
    print(k+1, res)
    best_k[k+1] = res


GLPSOL: GLPK LP/MIP Solver, v4.62
Parameter(s) specified in the command line:
 --cpxlp /var/folders/q8/0m2ycgr143349p67frdk52d80000gn/T/c6e27ce6d96640ecbe69f693dccca4ad-pulp.lp
 -o /var/folders/q8/0m2ycgr143349p67frdk52d80000gn/T/c6e27ce6d96640ecbe69f693dccca4ad-pulp.sol
Reading problem data from '/var/folders/q8/0m2ycgr143349p67frdk52d80000gn/T/c6e27ce6d96640ecbe69f693dccca4ad-pulp.lp'...
435 rows, 352 columns, 1337 non-zeros
352 integer variables, 128 of which are binary
1056 lines were read
GLPK Integer Optimizer, v4.62
435 rows, 352 columns, 1337 non-zeros
352 integer variables, 128 of which are binary
Preprocessing...
1 constraint coefficient(s) were reduced
347 rows, 277 columns, 1035 non-zeros
277 integer variables, 106 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.400e+01  ratio =  1.400e+01
GM: min|aij| =  5.399e-01  max|aij| =  1.852e+00  ratio =  3.431e+00
EQ: min|aij| =  3.152e-01  max|aij| =  1.000e+00  ratio =  3.173e+00
2N: min|aij| =  2.500e-01

KeyboardInterrupt: 

+809427: mip =   5.100000000e+01 <=   6.000000000e+01  17.6% (22979; 23867)
+829548: mip =   5.100000000e+01 <=   6.000000000e+01  17.6% (23422; 24410)
+847231: mip =   5.100000000e+01 <=   6.000000000e+01  17.6% (23835; 25051)
+866407: mip =   5.100000000e+01 <=   6.000000000e+01  17.6% (24295; 25647)
+886663: mip =   5.100000000e+01 <=   6.000000000e+01  17.6% (24774; 26218)
+906663: mip =   5.100000000e+01 <=   6.000000000e+01  17.6% (25317; 26689)
